# Writing To Vector DB

Write the manual qa (questions, answers) to Neo4j. 

In [1]:
!pip install azure-identity azure-keyvault-secrets

In [2]:
!pip install neomodel

In [3]:
!pip install openpyxl

In [4]:
!pip install tiktoken

In [6]:
!pip install openai

### Setup Keyvault

In [2]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

key_vault_name = "kv-bsauwmno"
kv_uri = f"https://{key_vault_name}.vault.azure.net/"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=kv_uri, credential=credential)

# Now you can use neo4j_url, neo4j_port, and neo4j_password in your application
neo4j_url = client.get_secret("NEO4JURL").value
neo4j_user = client.get_secret("NEO4JUSER").value
neo4j_password = client.get_secret("NEO4JPASSWORD").value
openai_key = client.get_secret("OPENAIKEY").value

# Define Neo4j Datamodel

Using neomodel

In [3]:
from neomodel.core import NodeClassAlreadyDefined
from neomodel import (
    config, 
    StructuredNode, 
    StringProperty, 
    IntegerProperty,
    UniqueIdProperty, 
    RelationshipTo, 
    ArrayProperty
)
from neomodel.contrib import SemiStructuredNode

try: 
    class WebPage(StructuredNode):
        webpage_id = IntegerProperty()
        url = StringProperty()

    class Answer(StructuredNode):
        answer_id = IntegerProperty()
        question_archetype = StringProperty()
        text = StringProperty()
       
        # String, Chunk doesn't exist yet. 
        # TODO: refine to CHUNK. 
        document_source = RelationshipTo(WebPage, 'HAS_DOCUMENT_SOURCE')

    class Question(StructuredNode):
        question_id = IntegerProperty()
        text = StringProperty()
        text_embedding = ArrayProperty()
        embedding_model = StringProperty()
        variation_category = StringProperty()

        answer = RelationshipTo(Answer, 'HAS_ANSWER')

except NodeClassAlreadyDefined as e: 
    print("Classes already defined.")

In [4]:
from neo4j import GraphDatabase
from neomodel import db

config.DATABASE_NAME = "neo4j"
config.DATABASE_URL = (
    f"bolt://{neo4j_user}:{neo4j_password}@" + 
    neo4j_url.replace('bolt://', '').replace('/:7687', ':7687/neo4j')
)

# Using URL - auto-managed
db.set_connection(url=config.DATABASE_URL)

# Create QA Dataset


In [5]:
import pandas as pd
from typing import List
from langchain.embeddings import OpenAIEmbeddings

embedding_model = "text-embedding-ada-002"

openai = OpenAIEmbeddings(
    openai_api_key=openai_key,
    model=embedding_model
)

question_df = (
    pd.read_excel(
        './data/qa_dataset.xlsx', 
        engine="openpyxl",
        sheet_name=1
    )
    .pipe(pd.DataFrame.dropna, subset=['question_id', 'question', 'answer_id', 'group'])
)

answer_df = pd.read_excel(
    './data/qa_dataset.xlsx', 
    engine="openpyxl",
    sheet_name=2
)

In [10]:
from typing import List

def embed_column(
        df: pd.DataFrame, 
        col_name: str, 
        col_name_embedded: str) -> pd.DataFrame:
    docs = list(df[col_name])
    embeddings: List[List[float]] = openai.embed_documents(texts=docs)
    df[col_name_embedded] = embeddings
    return df 

def cast_col(df: pd.DataFrame, col: str, cast_type: str) -> pd.DataFrame:
    df[col] = df[col].astype(cast_type)
    return df

embedded_questions_df = (
    question_df.copy(deep=True)
    .pipe(
        embed_column,
        col_name="question", 
        col_name_embedded="question_embedded"
    )
    .pipe(cast_col, col='answer_id', cast_type='int64')
    .pipe(cast_col, col='question_id', cast_type='int64')
)

# Create Derived Datasets

## Answer Nodes

Note: hardcoded to use 1 url for now. 

In [11]:
embedded_questions_df

,question_id,group,question,answer_id,question_embedded
0,1,time,Is het mogelijk om 8 uur na de geboorte met mi...,1,"[0.005267701044878656, -0.01916212564228965, 0..."
1,2,time,Kan ik met mijn pasgeboren baby 12 uur na de g...,1,"[0.003690805873760612, -0.014750647847488607, ..."
2,3,time,Is het toegestaan om 10 uur na de bevalling he...,1,"[0.009526361639722546, -0.028477197298947928, ..."
3,4,time,Mag ik 15 uur na de geboorte van mijn kind het...,1,"[0.015687218959545814, -0.011676282452501028, ..."
4,5,time,Kan ik 30 uur na de geboorte met mijn baby het...,1,"[0.011792143262032683, -0.019756075642396006, ..."
...,...,...,...,...,...
594,595,very_low_literacy,Hoe kan ik mijn baby gezond houden?,4,"[0.007664622577979124, -0.010371250813931438, ..."
595,596,very_low_literacy,Wat moet ik doen als mijn baby ziek wordt?,4,"[0.0021688599254414565, -0.012936521014286493,..."
596,597,very_low_literacy,Hoe kan ik ervoor zorgen dat mijn baby goed gr...,4,"[1.1692477838325705e-05, -0.021589238952175106..."
597,598,very_low_literacy,Wat als ik denk dat er iets mis is met mijn baby?,4,"[0.004506439697837641, -0.010990415202136084, ..."


In [6]:
import pandas as pd 

answers_df = (
    answer_df.copy(deep=True)
    .pipe(pd.DataFrame.rename, columns={
        "answer": "text",
        "question": "question_archetype"
    })
    # .pipe(pd.DataFrame.drop, ['question'], inplace=False, axis=1)
)
answers_df


,answer_id,question_archetype,text,source
0,1,Mag ik na 6u na de geboorte naar huis met mijn...,"Neen, een pasgeboren baby moet de eerste 24 uu...",https://www.azstlucas.be/onderzoek-en-behandel...
1,2,Zijn er specifieke gezondheidschecks voor de b...,Uw baby blijft eerste 24 uur na de geboorte in...,https://www.azstlucas.be/onderzoek-en-behandel...
2,3,Wat als mijn baby te vroeg geboren is? Wanneer...,Als je baby te vroeg geboren is (tussen 36-37 ...,https://www.azstlucas.be/onderzoek-en-behandel...
3,4,Welke problemen kunnen er zoal ontstaan na een...,Na de geboorte kunnen er problemen zoals aange...,https://www.azstlucas.be/onderzoek-en-behandel...


In [14]:
for _, obj in answers_df.iterrows():    
    a = Answer(
        answer_id=obj['question_id'],
        question_archetype=obj['question_archetype'],
        text=obj['text']
    )
    a.save()

## Answer to WebPage Relationship

In [15]:
# Not much different webpages for now. Single one. 

for _, obj in answers_df.iterrows():    
    answer = Answer.nodes.get(answer_id=obj['answer_id'])
    webpage = WebPage.nodes.get(url=obj['source'])
    answer.document_source.connect(webpage)

## Question Nodes

In [13]:
def add_model(df: pd.DataFrame, name: str) -> pd.DataFrame:
    df['model_name'] = name
    return df

questions_df = (
    embedded_questions_df.copy(deep=True)
    .pipe(pd.DataFrame.rename, columns={
        "group": "variation_category", 
        "question": "text",
        "question_embedded": "text_embedding", 
    })
    .pipe(add_model, name=embedding_model)
)
questions_df

,question_id,variation_category,text,answer_id,text_embedding,model_name
0,1,time,Is het mogelijk om 8 uur na de geboorte met mi...,1,"[0.005267701044878656, -0.01916212564228965, 0...",text-embedding-ada-002
1,2,time,Kan ik met mijn pasgeboren baby 12 uur na de g...,1,"[0.003690805873760612, -0.014750647847488607, ...",text-embedding-ada-002
2,3,time,Is het toegestaan om 10 uur na de bevalling he...,1,"[0.009526361639722546, -0.028477197298947928, ...",text-embedding-ada-002
3,4,time,Mag ik 15 uur na de geboorte van mijn kind het...,1,"[0.015687218959545814, -0.011676282452501028, ...",text-embedding-ada-002
4,5,time,Kan ik 30 uur na de geboorte met mijn baby het...,1,"[0.011792143262032683, -0.019756075642396006, ...",text-embedding-ada-002
...,...,...,...,...,...,...
594,595,very_low_literacy,Hoe kan ik mijn baby gezond houden?,4,"[0.007664622577979124, -0.010371250813931438, ...",text-embedding-ada-002
595,596,very_low_literacy,Wat moet ik doen als mijn baby ziek wordt?,4,"[0.0021688599254414565, -0.012936521014286493,...",text-embedding-ada-002
596,597,very_low_literacy,Hoe kan ik ervoor zorgen dat mijn baby goed gr...,4,"[1.1692477838325705e-05, -0.021589238952175106...",text-embedding-ada-002
597,598,very_low_literacy,Wat als ik denk dat er iets mis is met mijn baby?,4,"[0.004506439697837641, -0.010990415202136084, ...",text-embedding-ada-002


In [17]:
for _, obj in questions_df.iterrows():    
    q = Question(
        question_id=obj['question_id'], 
        text=obj['text'], 
        text_embedding=obj['text_embedding'],
        embedding_model=obj['model_name'],
        variation_category=obj['variation_category']
    )
    q.save()

# Question to Answer relationships

In [18]:
questions_df

,question_id,variation_category,text,answer_id,text_embedding,model_name
0,1,time,Is het mogelijk om 8 uur na de geboorte met mi...,1,"[0.005267701044878656, -0.01916212564228965, 0...",text-embedding-ada-002
1,2,time,Kan ik met mijn pasgeboren baby 12 uur na de g...,1,"[0.003690805873760612, -0.014750647847488607, ...",text-embedding-ada-002
2,3,time,Is het toegestaan om 10 uur na de bevalling he...,1,"[0.009526361639722546, -0.028477197298947928, ...",text-embedding-ada-002
3,4,time,Mag ik 15 uur na de geboorte van mijn kind het...,1,"[0.01572584320743565, -0.011663864069330074, 0...",text-embedding-ada-002
4,5,time,Kan ik 30 uur na de geboorte met mijn baby het...,1,"[0.011793966048119716, -0.019708915948374487, ...",text-embedding-ada-002
...,...,...,...,...,...,...
594,595,very_low_literacy,Hoe kan ik mijn baby gezond houden?,4,"[0.0077082431247685185, -0.010396150609807595,...",text-embedding-ada-002
595,596,very_low_literacy,Wat moet ik doen als mijn baby ziek wordt?,4,"[0.002168859925441457, -0.012936521014286495, ...",text-embedding-ada-002
596,597,very_low_literacy,Hoe kan ik ervoor zorgen dat mijn baby goed gr...,4,"[1.5926804948455913e-06, -0.0215753217245767, ...",text-embedding-ada-002
597,598,very_low_literacy,Wat als ik denk dat er iets mis is met mijn baby?,4,"[0.00448571759604563, -0.010967826476461223, -...",text-embedding-ada-002


In [19]:
for _, obj in questions_df.iterrows():
    answer = Answer.nodes.get(answer_id=obj['answer_id'])
    question = Question.nodes.get(question_id=obj['question_id'])
    question.answer.connect(answer)

# Manual Driver Queries



## Indexing

In [15]:
import neo4j
import pandas as pd
from typing import List 
from neo4j import GraphDatabase, RoutingControl

def to_pandas(records: List[object]) -> pd.DataFrame:
    return pd.DataFrame.from_records([dict(r) for r in records])

def add_fts_index(driver, label="Chunk", prop="text"):
    driver.execute_query(
        f"""
            CREATE FULLTEXT INDEX {"fts_" + label + "_" + prop} IF NOT EXISTS
            FOR (n:{label})
            ON EACH [n.{prop}]
        """,
        database_="neo4j",
    )

def get_indices(driver, db='neo4j'): 
    records, _, _ = driver.execute_query(
        "SHOW ALL INDEXES",
        database_= db, 
        routing_= RoutingControl.READ,
    )
    return to_pandas(records)

def get_chunks(driver, limit=20, db="neo4j") -> pd.DataFrame:
    records, _, _ = driver.execute_query(
        "MATCH (c:Chunk)"
        "RETURN c.text limit $limit",
        database_=db, 
        limit=limit,
        routing_= RoutingControl.READ,
    )
    return to_pandas(records)

def add_vector_index(
        driver, 
        label="Chunk", 
        prop="embedding", 
        idx_type="cosine", 
        vec_size=768): 
    driver.execute_query(
        f"""
            CALL db.index.vector.createNodeIndex(
                '{("vi_" + label + "_" + prop + "_" + idx_type + "_" + str(vec_size)).lower()}', 
                '{label}',
                '{prop}', 
                {vec_size}, 
                '{idx_type}'
            )
        """,
        database_="neo4j",
    ) 

# Drop index using: DROP INDEX `vi_question_text_embedding_cosine`
with GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password)) as driver:
    # add_fts_index(driver, label="Question", prop="text")
    # add_fts_index(driver, label="Answer", prop="text")
    add_vector_index(driver, label="Question", prop="text_embedding", vec_size=1536)
    index_df = get_indices(driver)

# df.head()
index_df

,id,name,state,populationPercent,type,entityType,labelsOrTypes,properties,indexProvider,owningConstraint,lastRead,readCount
0,7,fts_Answer_text,ONLINE,100.000000,FULLTEXT,NODE,[Answer],[text],fulltext-1.0,None,None,0.0
1,3,fts_Chunk_text,ONLINE,100.000000,FULLTEXT,NODE,[Chunk],[text],fulltext-1.0,None,2023-12-10T18:37:59.445000000+00:00,344.0
2,6,fts_Question_text,ONLINE,100.000000,FULLTEXT,NODE,[Question],[text],fulltext-1.0,None,None,0.0
3,1,index_343aff4e,ONLINE,100.000000,LOOKUP,NODE,None,None,token-lookup-1.0,None,2023-12-10T19:11:43.090000000+00:00,54583.0
4,2,index_f7700477,ONLINE,100.000000,LOOKUP,RELATIONSHIP,None,None,token-lookup-1.0,None,2023-12-10T17:47:32.485000000+00:00,5.0
5,4,vi_chunk_qa_embedding_cosine,ONLINE,100.000000,VECTOR,NODE,[Chunk],[qa_embedding],vector-1.0,None,None,0.0
6,5,vi_chunk_retrieval_embedding_cosine,ONLINE,100.000000,VECTOR,NODE,[Chunk],[retrieval_embedding],vector-1.0,None,None,0.0
7,8,vi_question_text_embedding_cosine_1536,POPULATING,11.031307,VECTOR,NODE,[Question],[text_embedding],vector-1.0,None,None,NaN
